In [1]:
import pandas as pd
import numpy
from matplotlib import pyplot as plt
import math
import csv

In [2]:
NP = 10**19
NB = 10**19
EiP_D = 0.045
EiB_A = 0.045
q = 1.602*10**(-19)
T = 300
k = 1.380649*10**(-23)
Eg = 1.126
Nv = 2.91*10**19
Nc = 1.09*10**19
diel = 12*8.8541878*10**(-12)

In [3]:
ni = numpy.float64((Nc*Nv)**0.5*math.exp(-Eg*q/2/k/T)) # расчет концентрации в собственном полупроводнике
print(ni)
NDn = numpy.float64(NP*math.exp(-EiP_D*q/(k*T))) # расчет ионизированных атомов фосфора в донорном состоянии
NAp = numpy.float64(NB*math.exp(-EiB_A*q/(k*T))) # расчет ионизированных атомов бора в акцепторном состоянии
print(NDn,NAp)

6218973741.560536
1.7543515088227694e+18 1.7543515088227694e+18


In [4]:
pp = numpy.float64((NAp+(NAp**2+4*ni**2)**0.5)/2) # расчет концентрации дырок в p-типе
np = numpy.float64(ni**2/pp) # расчет концентрации электронов в p-типе
nn = numpy.float64((NDn+(NDn**2+4*ni**2)**0.5)/2) # расчет концентрации электронов в n-типе
pn = numpy.float64(ni**2/nn) # расчет концентрации дырок в n-типе
print(pp,np,nn,pn)

1.7543515088227694e+18 22.045544580841813 1.7543515088227694e+18 22.045544580841813


In [5]:
Vni = numpy.float64(k*T/q*math.log(nn/ni))
Vip = numpy.float64(k*T/q*math.log(pp/ni))
print(Vni,Vip)
fni = numpy.float64(Vni*q/k/T)
fip = numpy.float64(Vip*q/k/T)
print(fni,fip)

0.5030774753415702 0.5030774753415702
19.457760215116114 19.457760215116114


In [204]:
size = 1001
x1 = 500
x2 = 501
h = numpy.zeros((size), dtype = 'float')
fi = numpy.zeros((size), dtype = 'float')
E = numpy.zeros((size), dtype = 'float')
C = numpy.zeros((size), dtype = 'float')
err = numpy.zeros((size), dtype = 'float')

In [205]:
for i in range(0,x1):
    fi[i] = 0
    h[i] = 10**(-8)
for i in range(x1,x2):
    fi[i] = 18.581268
    h[i] = 10**(-8)
for i in range(x2,size):
    fi[i] = 18.581268 + 18.743331
    h[i] = 10**(-8)

In [206]:
M = 1
a = q*q/diel/k/T/100
v = 0
print(a)
j = 0

5.831637376360708e-10


In [207]:
try:
    while j < 1000:
        for i in range(1,x1):
            v = (fi[i+1]+fi[i-1]+a*h[i]*h[i]*(-NAp+pp*math.exp(-fi[i])-pn*math.exp(fi[i])))/2
            d = abs(v-fi[i])
            err[i] = d
            fi[i] = v
        for i in range(x1,x2):
            v = (fi[i+1]+fi[i-1]+a*h[i]*h[i]*ni*(math.exp(-(fi[i]-fip))-math.exp((fi[i]-fip))))/2
            d = abs(v-fi[i])
            err[i] = d
            fi[i] = v
        for i in range(x2,size-1):
            v = (fi[i+1]+fi[i-1]+a*h[i]*h[i]*(NDn-nn*math.exp(fi[i]-fip-fni)+np*math.exp(-(fi[i]-fip-fni))))/2
            d = abs(v-fi[i])
            err[i] = d
            fi[i] = v
        j = j + 1
except:
    print(i,j)

In [208]:
for i in range (0,size-1):
    E[i] = -(fi[i + 1] - fi[i]) / h[i]
for i in range (1,size-1):
    C[i] = -(fi[i + 1] - 2 * fi[i] + fi[i - 1]) / (h[i] * h[i])

In [209]:
data = numpy.zeros((size,4), dtype = 'float')
for i in range (1,size):
    data[i,0] = data[i-1,0]+h[i]
    data[i,1] = C[i]
    data[i,2] = E[i]
    data[i,3] = fi[i]
print(data)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-08 -1.23977088e-08  4.33657911e-15 -4.46055620e-23]
 [ 2.00000000e-08 -1.21689397e-08  4.21488971e-15 -8.79713531e-23]
 ...
 [ 9.98000000e-06  8.00094853e+08  2.82497316e+02  3.73246047e+01]
 [ 9.99000000e-06  8.14634760e+08  2.90643663e+02  3.73246019e+01]
 [ 1.00000000e-05  0.00000000e+00  0.00000000e+00  3.73245990e+01]]


In [210]:
df = pd.DataFrame(data)

In [211]:
pd.DataFrame.to_csv(df, path_or_buf = r"C:\Users\kpebe\OneDrive\Рабочий стол\data.csv", sep = "	", index = False, header = True)

In [212]:
Co = 0
for i in range (0,size):
    if data[i,1] > 0:
        Co = Co + data[i,1]*h[i]
print(Co)

33308954.92979807
